# **Statistical analysis of salary in Data Jobs (2023)**
*** 

### Source - Kaggle
Dataset:
__[Data Science Salaries 2023 💸](https://www.kaggle.com/datasets/arnabchaki/data-science-salaries-2023?resource=download)__

### About Dataset
Data Job Salaries Dataset contains 11 columns, each are:

1. work_year: The year the salary was paid.

2. experience_level: The experience level in the job during the year
3. employment_type: The type of employment for the role
4. job_title: The role worked in during the year.
5. salary: The total gross salary amount paid.
6. salary_currency: The currency of the salary paid as an ISO 4217 currency code.
7. salaryinusd: The salary in USD
8. employee_residence: Employee's primary country of residence in during the work year as an ISO 3166 country code.
9. remote_ratio: The overall amount of work done remotely
10. company_location: The country of the employer's main office or contracting branch
11. company_size: The median number of people that worked for the company during the year


***
## **Data**

### **Needed packages**

In [2]:
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as se
from scipy import stats
import scikit_posthocs as sp
from statistics import median

### **Loading the data**

In [3]:
file = pd.read_csv(r'C:\Users\Jakub\Desktop\Projects\DATA\ds_salaries.csv', sep=',')
data = pd.DataFrame(file)
print(data)

      work_year experience_level employment_type                 job_title  \
0          2023               SE              FT  Principal Data Scientist   
1          2023               MI              CT               ML Engineer   
2          2023               MI              CT               ML Engineer   
3          2023               SE              FT            Data Scientist   
4          2023               SE              FT            Data Scientist   
...         ...              ...             ...                       ...   
3750       2020               SE              FT            Data Scientist   
3751       2021               MI              FT  Principal Data Scientist   
3752       2020               EN              FT            Data Scientist   
3753       2020               EN              CT     Business Data Analyst   
3754       2021               SE              FT      Data Science Manager   

       salary salary_currency  salary_in_usd employee_residence

### **Data structure**

In [4]:
for col in data:
    print(data[col].unique())

[2023 2022 2020 2021]
['SE' 'MI' 'EN' 'EX']
['FT' 'CT' 'FL' 'PT']
['Principal Data Scientist' 'ML Engineer' 'Data Scientist'
 'Applied Scientist' 'Data Analyst' 'Data Modeler' 'Research Engineer'
 'Analytics Engineer' 'Business Intelligence Engineer'
 'Machine Learning Engineer' 'Data Strategist' 'Data Engineer'
 'Computer Vision Engineer' 'Data Quality Analyst'
 'Compliance Data Analyst' 'Data Architect'
 'Applied Machine Learning Engineer' 'AI Developer' 'Research Scientist'
 'Data Analytics Manager' 'Business Data Analyst' 'Applied Data Scientist'
 'Staff Data Analyst' 'ETL Engineer' 'Data DevOps Engineer' 'Head of Data'
 'Data Science Manager' 'Data Manager' 'Machine Learning Researcher'
 'Big Data Engineer' 'Data Specialist' 'Lead Data Analyst'
 'BI Data Engineer' 'Director of Data Science'
 'Machine Learning Scientist' 'MLOps Engineer' 'AI Scientist'
 'Autonomous Vehicle Technician' 'Applied Machine Learning Scientist'
 'Lead Data Scientist' 'Cloud Database Engineer' 'Financial D

In [5]:
for col in data:
    print(col,':',len(data[col].unique()))

work_year : 4
experience_level : 4
employment_type : 4
job_title : 93
salary : 815
salary_currency : 20
salary_in_usd : 1035
employee_residence : 78
remote_ratio : 3
company_location : 72
company_size : 3


Amount of unique values in columns about salary was expected, but we can see that amount of job titles is very big and we can easily classify them to smaller groups.

In [6]:
data['job_title'].unique()

array(['Principal Data Scientist', 'ML Engineer', 'Data Scientist',
       'Applied Scientist', 'Data Analyst', 'Data Modeler',
       'Research Engineer', 'Analytics Engineer',
       'Business Intelligence Engineer', 'Machine Learning Engineer',
       'Data Strategist', 'Data Engineer', 'Computer Vision Engineer',
       'Data Quality Analyst', 'Compliance Data Analyst',
       'Data Architect', 'Applied Machine Learning Engineer',
       'AI Developer', 'Research Scientist', 'Data Analytics Manager',
       'Business Data Analyst', 'Applied Data Scientist',
       'Staff Data Analyst', 'ETL Engineer', 'Data DevOps Engineer',
       'Head of Data', 'Data Science Manager', 'Data Manager',
       'Machine Learning Researcher', 'Big Data Engineer',
       'Data Specialist', 'Lead Data Analyst', 'BI Data Engineer',
       'Director of Data Science', 'Machine Learning Scientist',
       'MLOps Engineer', 'AI Scientist', 'Autonomous Vehicle Technician',
       'Applied Machine Learning Sc

#### Changing structure of job_title column

The most known classification for jobs around Data are:

- **Data Scientis**
- **Data Engineer**
- **Data Analyst**

For time sake I asked *ChatGPT* to classify the job titles. It wasn't great, but after checking and some adjustments I managed to get pretty good classification.

##### Adding new column 'job_type' (duplicate of 'job_title')

In [7]:
data['job_type'] = data['job_title']
print(data.columns)

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size', 'job_type'],
      dtype='object')


In [8]:
data = data[['work_year',
 'experience_level',
 'employment_type',
 'job_title',
 'job_type',
 'salary',
 'salary_currency',
 'salary_in_usd',
 'employee_residence',
 'remote_ratio',
 'company_location',
 'company_size']]

print(data.columns)

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'job_type', 'salary', 'salary_currency', 'salary_in_usd',
       'employee_residence', 'remote_ratio', 'company_location',
       'company_size'],
      dtype='object')


##### Making remaping dictionaries

In [9]:
remap_science = {'job_type' : ['Data Scientist', 'Principal Data Scientist', 'Applied Scientist', 'AI Developer',
        'Research Scientist', 'Head of Data', 'Data Science Manager', 'AI Programmer',  
        'Director of Data Science', 'Machine Learning Scientist', 'Applied Machine Learning Scientist',
        'Lead Data Scientist', 'Deep Learning Researcher', 'Data Science Consultant', 
        'Machine Learning Developer', '3D Computer Vision Researcher', 'Machine Learning Researcher',
        'Data Science Tech Lead', 'Data Scientist Lead', 'Product Data Scientist', 'Data Science Lead',
        'Machine Learning Manager', 'AI Scientist', 'Head of Data Science', 'Applied Data Scientist',
        'Head of Machine Learning','Staff Data Scientist']}

remap_engineer = {'job_type' : ['Data Engineer', 'Data Modeler', 'Analytics Engineer', 'Business Intelligence Engineer',
        'Data Strategist', 'Data DevOps Engineer', 'Big Data Engineer', 'Data Specialist',
        'BI Data Engineer', 'Data Infrastructure Engineer', 'Cloud Database Engineer', 'ETL Engineer',
        'Data Operations Engineer', 'BI Developer', 'Azure Data Engineer', 'Computer Vision Engineer',
        'Machine Learning Infrastructure Engineer', 'Cloud Data Engineer', 'ETL Developer',
        'Data Architect', 'Big Data Architect', 'Autonomous Vehicle Technician', 'ML Engineer',
        'Machine Learning Software Engineer', 'Data Analytics Engineer', 'Research Engineer',
        'Computer Vision Software Engineer', 'Data Lead', 'Data Management Specialist',
        'Applied Machine Learning Engineer', 'MLOps Engineer', 'Machine Learning Research Engineer',
        'Deep Learning Engineer', 'Machine Learning Engineer', 'Data Science Engineer',
        'Lead Machine Learning Engineer', 'NLP Engineer', 'Principal Machine Learning Engineer',
        'Software Data Engineer', 'Principal Data Architect', 'Lead Data Engineer','Marketing Data Engineer',
        'Cloud Data Architect','Principal Data Engineer']}

remap_analyst = {'job_type' : ['Data Analyst', 'Data Quality Analyst', 'Compliance Data Analyst', 'Business Data Analyst',
        'Lead Data Analyst', 'Marketing Data Analyst', 'Data Analytics Specialist', 'Insight Analyst',
        'Product Data Analyst', 'BI Data Analyst', 'Data Operations Analyst', 'Data Analytics Lead',
        'Principal Data Analyst', 'Financial Data Analyst', 'BI Analyst', 'Data Analytics Manager',
        'Data Analytics Consultant', 'Data Manager', 'Manager Data Management','Staff Data Analyst',
        'Power BI Developer','Finance Data Analyst']}

##### ReMaping job titles in 'job_type' column

In [10]:
data.replace(remap_science, value='Data Scientis', inplace=True)
data.replace(remap_engineer, value='Data Engineer', inplace=True)
data.replace(remap_analyst, value='Data Analyst', inplace=True)

In [11]:
print(data)

      work_year experience_level employment_type                 job_title  \
0          2023               SE              FT  Principal Data Scientist   
1          2023               MI              CT               ML Engineer   
2          2023               MI              CT               ML Engineer   
3          2023               SE              FT            Data Scientist   
4          2023               SE              FT            Data Scientist   
...         ...              ...             ...                       ...   
3750       2020               SE              FT            Data Scientist   
3751       2021               MI              FT  Principal Data Scientist   
3752       2020               EN              FT            Data Scientist   
3753       2020               EN              CT     Business Data Analyst   
3754       2021               SE              FT      Data Science Manager   

           job_type   salary salary_currency  salary_in_usd  \


##### Checking if everything went correct

In [12]:
data['job_type'].unique()

array(['Data Scientis', 'Data Engineer', 'Data Analyst'], dtype=object)

##### Changing structure of experience_level for more clear understunding

In [13]:
data['experience_level'].unique()

array(['SE', 'MI', 'EN', 'EX'], dtype=object)

In [14]:
data.replace('SE', value='Senior', inplace=True)
data.replace('MI', value='Mid', inplace=True)
data.replace('EN', value='Entry', inplace=True)
data.replace('EX', value='Expert', inplace=True)

In [15]:
data['experience_level'].unique()

array(['Senior', 'Mid', 'Entry', 'Expert'], dtype=object)

***
## **Statistical analysis**

### **Salary distribution by job category (in USD)**

In [16]:
mDS = median(data['salary_in_usd'].loc[(data['work_year'] == 2023) & (data['job_type'] == 'Data Scientis')])
mDA = median(data['salary_in_usd'].loc[(data['work_year'] == 2023) & (data['job_type'] == 'Data Analyst')])
mDE = median(data['salary_in_usd'].loc[(data['work_year'] == 2023) & (data['job_type'] == 'Data Engineer')])

In [17]:
print('DS_median:',mDS,'\n',
      'DA_median:',mDA,'\n',
      'DE_median:',mDE)

DS_median: 156400.0 
 DA_median: 108000.0 
 DE_median: 147100


In [18]:
import plotly.express as px
fig = px.histogram(data.loc[data['work_year']==2023], x='salary_in_usd', nbins=50, color='job_type', marginal='rug', opacity=0.5)

fig.add_vline(x=mDS, line_width=3, line_dash="dash", line_color="blue")
fig.add_vline(x=mDA, line_width=3, line_dash="dash", line_color="green")
fig.add_vline(x=mDE, line_width=3, line_dash="dash", line_color="red")

fig.show()

### **Analyzing differences in salary in 2023 for different experiecne levels (between all job types)**

##### Picking data

In [19]:
En2023 = data['salary_in_usd'].loc[(data['work_year'] == 2023) & (data['experience_level'] == 'Entry')]
Mi2023 = data['salary_in_usd'].loc[(data['work_year'] == 2023) & (data['experience_level'] == 'Mid')]
Se2023 = data['salary_in_usd'].loc[(data['work_year'] == 2023) & (data['experience_level'] == 'Senior')]
Ex2023 = data['salary_in_usd'].loc[(data['work_year'] == 2023) & (data['experience_level'] == 'Expert')]

##### Shapiro test for normal distribution

$H_0$: The data follows a normal distribution.

$H_1$:  The data does not follow a normal distribution.

In [20]:
stat_shapiro , p_value = stats.shapiro(En2023)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

print('_______________________________________')

stat_shapiro , p_value = stats.shapiro(Mi2023)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

print('_______________________________________')

stat_shapiro , p_value = stats.shapiro(Se2023)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

print('_______________________________________')

stat_shapiro , p_value = stats.shapiro(Ex2023)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

Shapiro-statistic value:  0.966701865196228
P-Value:  0.005015839822590351
_______________________________________
Shapiro-statistic value:  0.9666078090667725
P-Value:  9.931359272741247e-07
_______________________________________
Shapiro-statistic value:  0.9768952131271362
P-Value:  1.7044455040068907e-13
_______________________________________
Shapiro-statistic value:  0.961067795753479
P-Value:  0.05306477099657059


Only one of the groups comes from normal distribution, that's why we are not going to compare the variances between groups and perform Kruskal-Wallis test (Non-parametric ANOVA).

##### Kruskala Wallis test

$H_0$: There are no differences between the populations' medians. ($u_1=u_2=...=u_n$)

$H_1$: At least one population median is different from the others. ($u_i\not=u_j$)

In [21]:
krus, p_value = stats.kruskal(En2023, Mi2023, Se2023, Ex2023)
print("Kruskal-statistic value: ", krus)  
print("P-Value: ", p_value)

Kruskal-statistic value:  270.9918874965717
P-Value:  1.883091179518999e-58


Our p-value is below .05 thats why we are rejecting the null hypothesis for alternative hypothesis. There are significant statistical differences between groups.

##### Post-hoc (Dunn test)

Therefore, after picking alternative hypothesis  for the Kruskal-Wallis test, conducting non-parametric post-hoc tests is essential to determine the specific group differences.

In [22]:
df = [list(En2023), list(Mi2023), list(Se2023), list(Ex2023)]
round(sp.posthoc_dunn(df,p_adjust = 'bonferroni'),9)

,1,2,3,4
1,1.000000,0.024115,0.000000,0.000000
2,0.024115,1.000000,0.000000,0.000000
3,0.000000,0.000000,1.000000,0.000024
4,0.000000,0.000000,0.000024,1.000000


The results show that all the compirasion shows significant differense between analized groups (p-value below .05).

We are ignoring the diagonal, and the other half of the matrix because it is symmetrical.

##### Box-plot

In [23]:
import plotly.express as px
px.box(data.loc[(data['work_year'] == 2023)], x='salary_in_usd', y='experience_level', color='experience_level')

### **Analyzing differences in salary in 2023 for different experience levels based on data type job**

The previous analysis showed us the differences in earnings among different levels of experience in 2023. However, this analysis did not reveal how these differences look like across different types of data-related jobs. Therefore, we can conduct a more in-depth analysis by examining the differences for each type of job separately.

##### Picking data

In [24]:
En2023 = data[(data['work_year'] == 2023) & (data['experience_level'] == 'Entry')]
Mi2023 = data[(data['work_year'] == 2023) & (data['experience_level'] == 'Mid')]
Se2023 = data[(data['work_year'] == 2023) & (data['experience_level'] == 'Senior')]
Ex2023 = data[(data['work_year'] == 2023) & (data['experience_level'] == 'Expert')]

#### **Data Science**

In [25]:
En2023_DS = En2023['salary_in_usd'].loc[(En2023['job_type'] == 'Data Scientis')]
Mi2023_DS = Mi2023['salary_in_usd'].loc[(Mi2023['job_type'] == 'Data Scientis')]
Se2023_DS = Se2023['salary_in_usd'].loc[(Se2023['job_type'] == 'Data Scientis')]
Ex2023_DS = Ex2023['salary_in_usd'].loc[(Ex2023['job_type'] == 'Data Scientis')]

print('En:',len(En2023_DS),
      'Mi:',len(Mi2023_DS),
      'Se:',len(Se2023_DS),
      'Ex:',len(Ex2023_DS))


En: 31 Mi: 77 Se: 412 Ex: 18


##### Shapiro test for normal distribution

$H_0$: The data follows a normal distribution.

$H_1$:  The data does not follow a normal distribution.

In [26]:
stat_shapiro , p_value = stats.shapiro(En2023_DS)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

print('_______________________________________')

stat_shapiro , p_value = stats.shapiro(Mi2023_DS)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

print('_______________________________________')

stat_shapiro , p_value = stats.shapiro(Se2023_DS)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

print('_______________________________________')

stat_shapiro , p_value = stats.shapiro(Ex2023_DS)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

Shapiro-statistic value:  0.9449013471603394
P-Value:  0.11280075460672379
_______________________________________
Shapiro-statistic value:  0.9444264769554138
P-Value:  0.0021267442498356104
_______________________________________
Shapiro-statistic value:  0.9836085438728333
P-Value:  0.0001293498935410753
_______________________________________
Shapiro-statistic value:  0.9462328553199768
P-Value:  0.36901190876960754


Only two of the groups comes from normal distribution, that's why we still need to perform Kruskal-Wallis test (Non-parametric ANOVA).

##### Kruskal-Wallis test

$H_0$: There are no differences between the populations' medians. ($u_1=u_2=...=u_n$)

$H_1$: At least one population median is different from the others. ($u_i\not=u_j$)

In [27]:
krus, p_valuea = stats.kruskal(En2023_DS, Mi2023_DS, Se2023_DS, Ex2023_DS)
print("Kruskal-statistic value: ", krus)  
print("P-Value: ", p_value)

Kruskal-statistic value:  75.29163530782296
P-Value:  0.36901190876960754


The result of Kruskal-Wallis test shows that there is no significant differences between groups (p-value above .05).

*There is no need for post-hoc test.*

##### Box-plot

In [28]:
import plotly.express as px
px.box(data[(data['work_year'] == 2023) & (data['job_type'] == 'Data Scientis')],
        x='salary_in_usd', y='experience_level', color='experience_level')

#### **Data Analytics**

In [29]:
En2023_DA = En2023['salary_in_usd'].loc[(En2023['job_type'] == 'Data Analyst')]
Mi2023_DA = Mi2023['salary_in_usd'].loc[(Mi2023['job_type'] == 'Data Analyst')]
Se2023_DA = Se2023['salary_in_usd'].loc[(Se2023['job_type'] == 'Data Analyst')]
Ex2023_DA = Ex2023['salary_in_usd'].loc[(Ex2023['job_type'] == 'Data Analyst')]

print('En:',len(En2023_DA),
      'Mi:',len(Mi2023_DA),
      'Se:',len(Se2023_DA),
      'Ex:',len(Ex2023_DA))

En: 36 Mi: 92 Se: 236 Ex: 2


Based on the measured group sizes, we can observe that the number of data on earnings of individuals in the Data Analyst department in 2023 is very small (n=2). To avoid distortions in the analysis, we will not consider it in further analysis.

##### Shapiro test for normal distribution

$H_0$: The data follows a normal distribution.

$H_1$:  The data does not follow a normal distribution.

In [30]:
stat_shapiro , p_value = stats.shapiro(En2023_DA)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

print('_______________________________________')

stat_shapiro , p_value = stats.shapiro(Mi2023_DA)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

print('_______________________________________')

stat_shapiro , p_value = stats.shapiro(Se2023_DA)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

Shapiro-statistic value:  0.9605507850646973
P-Value:  0.22353383898735046
_______________________________________
Shapiro-statistic value:  0.9872921109199524
P-Value:  0.5187916159629822
_______________________________________
Shapiro-statistic value:  0.9537356495857239
P-Value:  7.2708144216449e-07


##### Kruskal-Wallis test

$H_0$: There are no differences between the populations' medians. ($u_1=u_2=...=u_n$)

$H_1$: At least one population median is different from the others. ($u_i\not=u_j$)

In [31]:
krus, p_valuea = stats.kruskal(En2023_DA, Mi2023_DA, Se2023_DA)
print("Kruskal-statistic value: ", krus)  
print("P-Value: ", p_value)

Kruskal-statistic value:  61.74969260712051
P-Value:  7.2708144216449e-07


##### Post-hoc (Dunn test)

In [32]:
df = [list(En2023_DA), list(Mi2023_DA), list(Se2023_DA)]
round(sp.posthoc_dunn(df,p_adjust = 'bonferroni'),9)

,1,2,3
1,1.000000,0.000045,0.000000
2,0.000045,1.000000,0.000275
3,0.000000,0.000275,1.000000


The results show significant differenses in all the compirasions between groups (p-value below .05).

##### Box-plot

In [33]:
import plotly.express as px
px.box(data[(data['work_year'] == 2023) & (data['job_type'] == 'Data Analyst')],
        x='salary_in_usd', y='experience_level', color='experience_level')

#### **Data Engineer**

In [34]:
En2023_DE = En2023['salary_in_usd'].loc[(En2023['job_type'] == 'Data Engineer')]
Mi2023_DE = Mi2023['salary_in_usd'].loc[(Mi2023['job_type'] == 'Data Engineer')]
Se2023_DE = Se2023['salary_in_usd'].loc[(Se2023['job_type'] == 'Data Engineer')]
Ex2023_DE = Ex2023['salary_in_usd'].loc[(Ex2023['job_type'] == 'Data Engineer')]

print('En:',len(En2023_DE),
      'Mi:',len(Mi2023_DE),
      'Se:',len(Se2023_DE),
      'Ex:',len(Ex2023_DE))

En: 51 Mi: 151 Se: 639 Ex: 40


##### Shapiro test for normal distribution

$H_0$: The data follows a normal distribution.

$H_1$:  The data does not follow a normal distribution.

In [35]:
stat_shapiro , p_value = stats.shapiro(En2023_DE)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

print('_______________________________________')

stat_shapiro , p_value = stats.shapiro(Mi2023_DE)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

print('_______________________________________')

stat_shapiro , p_value = stats.shapiro(Se2023_DE)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

print('_______________________________________')

stat_shapiro , p_value = stats.shapiro(Ex2023_DE)
print("Shapiro-statistic value: ", stat_shapiro)  
print("P-Value: ", p_value)

Shapiro-statistic value:  0.9594553112983704
P-Value:  0.07906678318977356
_______________________________________
Shapiro-statistic value:  0.9627605676651001
P-Value:  0.00042166205821558833
_______________________________________
Shapiro-statistic value:  0.9699187278747559
P-Value:  3.5741481996254265e-10
_______________________________________
Shapiro-statistic value:  0.947640061378479
P-Value:  0.06291612237691879


##### Kruskal-Wallis test

$H_0$: There are no differences between the populations' medians. ($u_1=u_2=...=u_n$)

$H_1$: At least one population median is different from the others. ($u_i\not=u_j$)

In [36]:
krus, p_valuea = stats.kruskal(En2023_DE, Mi2023_DE, Se2023_DE, Ex2023_DE)
print("Kruskal-statistic value: ", krus)  
print("P-Value: ", p_value)

Kruskal-statistic value:  111.50613995321928
P-Value:  0.06291612237691879


The result of Kruskal-Wallis test shows the same result like in Data Science department, that there is no significant differences between groups (p-value above .05).

*There is no need for post-hoc test.*

##### Box-plot

In [37]:
import plotly.express as px
px.box(data[(data['work_year'] == 2023) & (data['job_type'] == 'Data Engineer')],
        x='salary_in_usd', y='experience_level', color='experience_level')

#### Conclusion

After performed analysis, we had managed to see that significant differences were found only within the Data Analyst group, indicating that experience level has an bigger impact on earnings tha in other two groups. Further exploration of factors influencing earnings within the Data Analyst category may provide valuable insights.

### **Remote ratio over years**

In [38]:
data['remote_ratio'].unique()

array([100,   0,  50], dtype=int64)

In [39]:
data.pivot_table(index = ['remote_ratio'], aggfunc ='size')

remote_ratio
0      1923
50      189
100    1643
dtype: int64

In [40]:
rr_2020 = data[(data['work_year'] == 2020)].pivot_table(index = ['remote_ratio'], aggfunc ='size')
rr_2021 = data[(data['work_year'] == 2021)].pivot_table(index = ['remote_ratio'], aggfunc ='size')
rr_2022 = data[(data['work_year'] == 2022)].pivot_table(index = ['remote_ratio'], aggfunc ='size')
rr_2023 = data[(data['work_year'] == 2023)].pivot_table(index = ['remote_ratio'], aggfunc ='size')

In [41]:
pd.concat([rr_2020, rr_2021, rr_2022, rr_2023], axis=1).set_axis(['2020', '2021', '2022', '2023'], axis = 1)

,2020,2021,2022,2023
remote_ratio,,,,
0,16,34,711,1162
50,21,76,62,30
100,39,120,891,593


### **Analyzing dependence between remote ratio and job type**

#### **Analyzing all possible remote ratio levels**

##### Contingency Table

In [42]:
ct = pd.crosstab(data['remote_ratio'].loc[(data['work_year'] == 2023)], 
                 data['job_type'].loc[(data['work_year'] == 2023)], margins=True)
ct

job_type,Data Analyst,Data Engineer,Data Scientis,All
remote_ratio,,,,
0,231,574,357,1162
50,7,5,18,30
100,128,302,163,593
All,366,881,538,1785


##### Performing Chi-square Statistic

$H_0:$ There is no significant association between the variables.

$H_1:$ There is a significant association between the variables.

In [43]:
obs = np.array([ct.iloc[0][0:4].values,
                  ct.iloc[1][0:4].values])
stats.chi2_contingency(obs)[0:3]

print('\n',
      'Chi-square statistic:',stats.chi2_contingency(obs)[0],'\n',
      'P-Value:',stats.chi2_contingency(obs)[1],'\n',
      'Degree of Freedom:',stats.chi2_contingency(obs)[2])


 Chi-square statistic: 14.59286383171618 
 P-Value: 0.0021997989163488295 
 Degree of Freedom: 3


The conducted chi-square test of independence, examine the relationship between the amount of remote work hours (100, 50, or 0) and job type (Data Analyst, Data Engineer, and Data Scientist). The results of the test revealed a significant association between these two variables, with a p-value below .05. 

Obtained results shows that job type influences the amount of remote work hours, although it is not specified which category has more or fewer hours.

##### Box-plot

In [44]:
import plotly.express as px
px.box(data.loc[(data['work_year'] == 2023)], y='salary_in_usd', x='remote_ratio', color='remote_ratio')

#### **Analyzing only *100%* and *0%*  remote ratio levels**

In [45]:
data_n50 = data.loc[(data['remote_ratio'] == 100) | (data['remote_ratio'] == 0)]

##### Contingency Table

In [46]:
ct = pd.crosstab(data_n50['remote_ratio'].loc[(data['work_year'] == 2023)],
                 data_n50['job_type'].loc[(data['work_year'] == 2023)], margins=True)
ct

job_type,Data Analyst,Data Engineer,Data Scientis,All
remote_ratio,,,,
0,231,574,357,1162
100,128,302,163,593
All,359,876,520,1755


##### Performing Chi-square Statistic

$H_0:$ There is no significant association between the variables.

$H_1:$ There is a significant association between the variables.

In [47]:
obs = np.array([ct.iloc[0][0:4].values,
                  ct.iloc[1][0:4].values])
stats.chi2_contingency(obs)[0:3]

print('\n',
      'Chi-square statistic:',stats.chi2_contingency(obs)[0],'\n',
      'P-Value:',stats.chi2_contingency(obs)[1],'\n',
      'Degree of Freedom:',stats.chi2_contingency(obs)[2])


 Chi-square statistic: 2.129744850581266 
 P-Value: 0.5459186554567952 
 Degree of Freedom: 3


The conducted chi-square test of independence without 50% level of remote ratio, revealed that there is no significant association between these two variables, with a p-value above .05.

P-value so close to 1 shows very strong independence between those two variables.

##### Box-plot

In [48]:
import plotly.express as px
px.box(data_n50.loc[(data['work_year'] == 2023)], y='salary_in_usd', x='remote_ratio', color='remote_ratio')

***

## **Conclusion**

* Data Scientists and Data Engineers have comparable salaries between all level of experience
* Data Analysts have significantly different salaries based on experience
* The remote ratio does not seem to influence the anual salary unless we take 50% ratio into account